In [1]:
import geojson
import folium

# Таблица с координатами и радиусами
data = [
    {'label': 'poin1','lat': 55.806690, 'lon': 37.577019, 'radius': 2},
    {'label': 'poin2','lat': 55.741420, 'lon': 37.693748, 'radius': 2}
]



# Создание списка точек
points = []
for item in data:
    point = geojson.Point((item['lon'], item['lat']))
    points.append(point)

# Создание списка кругов
circles = []
for item in data:
    circle = geojson.Feature(
        geometry=geojson.Point((item['lon'], item['lat'])),
        properties={
            'radius': item['radius'],
            'color': 'red',
            'fillOpacity': 0.1,
            'stroke': 'red',
            'strokeWidth': 0.3,
            'label': item['label']
        }
    )
    circles.append(circle)

# Создание GeoJSON
geojson_data = geojson.FeatureCollection(circles)

# Сохранение GeoJSON в файл
with open('circles.geojson', 'w') as f:
    geojson.dump(geojson_data, f)

# Создание карты с кругами
m = folium.Map(location=[55.7558, 37.6173], zoom_start=12)
for item in data:
    folium.Circle(
        location=[item['lat'], item['lon']],
        radius=item['radius']*1000,  # радиус в метрах
        color='red',
        fill=True,
        stroke=False,
        popup=item['label']
        # fillOpacity=0.1,
        # stroke=True,
        # strokeWeight=2,
    ).add_to(m)

# Сохранение карты в файл
m.save('circles.html')